# Libraries

In [1]:
import argparse
import os
import numpy as np
import math
import sys
from pathlib import Path

import torchvision.transforms as T
from torchvision.utils import save_image
from PIL import Image

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

from IPython.display import clear_output

from google.colab import drive

# Training parameters

In [2]:
# makind directory for image samples while training
os.makedirs("images2", exist_ok=True)

n_epochs = 1000                            # number of epochs of training
batch_size = 64                            # size of the batches
lr = 0.0002                                # adam: learning rate
b1 = 0.5                                   # adam: decay of first order momentum of gradient
b2 = 0.999                                 # adam: decay of first order momentum of gradient
n_cpu = 8                                  # number of cpu threads to use during batch generation
latent_dim = 200                           # dimensionality of the latent space
img_size = 64                              # size of each image dimension
channels = 3                               # number of image channels
n_critic = 5                               # number of training steps for discriminator per iter
lip_value = 0.01                           # lower and upper clip value for disc. weights
sample_interval = 100                      # interval betwen image samples
img_shape = (channels, img_size, img_size) # shape of training sample

cuda = True if torch.cuda.is_available() else False

# Prepare data

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!unzip drive/MyDrive/resized.zip -d resized2
clear_output()

In [5]:
class HousesDatasetGAN(Dataset):
    '''our custom version of torch dataset for the pictures of houses'''

    def __init__(self, dataset_dir, transforms=None):
        '''save the paths to images of houses and transforms if needed'''
        self.houses_paths = list(map(str, Path(dataset_dir).rglob('*.jpg')))
        self.transforms   = transforms

    def __len__(self):
        '''size of list of paths'''
        return len(self.houses_paths)

    def __getitem__(self, idx):
        '''retrieve an image from path and return it's tranformed version'''

        # open the image in 3 channels 
        image = Image.open(self.houses_paths[idx]).convert('RGB')

        # transform it if needed
        if self.transforms is not None:
            image = self.transforms(image)

        # return in image and indicator that it is a real one
        return image, 1

In [12]:
# preparing the augmentations 
transforms = T.Compose([
    # T.RandomRotation(degrees=45),       #  <-- maybe it's a bad idea
    # T.GaussianBlur(kernel_size=(3, 3)), #  <-- same thing
    T.RandomHorizontalFlip(),
    # T.ColorJitter(brightness=0.4, hue=0.3),
    T.ToTensor(),
    T.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# creating a train dataset
houses_dataset = HousesDatasetGAN('resized2', transforms)

# Configure data loader
dataloader = torch.utils.data.DataLoader(
    houses_dataset,
    batch_size=batch_size,
    shuffle=True,
)

print(len(houses_dataset))

6497


# Models

## Generator

In [7]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 256, normalize=False),
            *block(256, 1024),
            *block(1024, 4096),
            *block(4096, 16384),
            nn.Linear(16384, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img

## Discriminator

In [8]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 16384),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(16384, 4096),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(4096, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

# Training

In [9]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [13]:
# Loss weight for gradient penalty
lambda_gp = 10

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [14]:
batches_done = 0
for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z)

        # Real images
        real_validity = discriminator(real_imgs)
        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i % n_critic == 0:

            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()

            if batches_done % sample_interval == 0:
                save_image(fake_imgs.data[:25], "images2/%d.png" % batches_done, nrow=5, normalize=True)

            batches_done += n_critic

    print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
        % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
    )

[Epoch 0/1000] [Batch 101/102] [D loss: -13.722965] [G loss: 11.760313]
[Epoch 1/1000] [Batch 101/102] [D loss: -19.565790] [G loss: 6.918622]
[Epoch 2/1000] [Batch 101/102] [D loss: -20.401299] [G loss: 6.548736]
[Epoch 3/1000] [Batch 101/102] [D loss: -14.634324] [G loss: 6.032339]
[Epoch 4/1000] [Batch 101/102] [D loss: -19.619642] [G loss: 17.022682]
[Epoch 5/1000] [Batch 101/102] [D loss: -18.874287] [G loss: 8.418646]
[Epoch 6/1000] [Batch 101/102] [D loss: -17.798012] [G loss: 21.639175]
[Epoch 7/1000] [Batch 101/102] [D loss: -10.534840] [G loss: 3.523641]
[Epoch 8/1000] [Batch 101/102] [D loss: -13.618046] [G loss: 12.221556]
[Epoch 9/1000] [Batch 101/102] [D loss: -10.235277] [G loss: 8.307076]
[Epoch 10/1000] [Batch 101/102] [D loss: -15.042346] [G loss: 11.215281]
[Epoch 11/1000] [Batch 101/102] [D loss: -6.928462] [G loss: -4.808434]
[Epoch 12/1000] [Batch 101/102] [D loss: -10.057541] [G loss: 3.400860]
[Epoch 13/1000] [Batch 101/102] [D loss: -11.281293] [G loss: 12.5065

KeyboardInterrupt: ignored

In [15]:
torch.save(discriminator, 'drive/MyDrive/d_big_last.pt')
torch.save(generator, 'drive/MyDrive/g_big_last.pt')

In [17]:
z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
gen_imgs = generator(z)
save_image(gen_imgs.data[:32], "images2/lastN.png", nrow=8, normalize=True)

# References:

* https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/wgan_gp/wgan_gp.py